In [1]:
#personal project - basketball detection openCV

import cv2
import imutils
import numpy as np
from collections import deque

def empty(arg):
    pass

cap = cv2.VideoCapture('/Users/anthonycerelli/Desktop/spinFinger.mp4') 

frame_width = int(cap.get(3)) 
frame_height = int(cap.get(4))

size = (frame_width, frame_height)

result = cv2.VideoWriter('spinFinal.avi',  
                         cv2.VideoWriter_fourcc(*'MJPG'), 
                         10, size)

# pts = deque(maxlen=args["buffer"])

# lower_orange = (7,180,180)
# upper_orange = (11, 255, 255)

while True: #loop to capture image by iteration through video
    
    success, img = cap.read() #success is a boolean that will be true / false
    if img is None:
        break
    output = img.copy()
    lower_orange = (0, 110, 150) #0 115 165
    upper_orange = (10, 255, 255)
    
    blurred = cv2.GaussianBlur(img, (11, 11), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    
    mask = cv2.inRange(hsv, lower_orange, upper_orange)
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)
    
    
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    center = None
    # only proceed if at least one contour was found
    if len(cnts) > 0:
        # find the largest contour in the mask, then use
        # it to compute the minimum enclosing circle and
        # centroid
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
        # only proceed if the radius meets a minimum size
        if radius > 10:
            # draw the circle and centroid on the frame,
            # then update the list of tracked points
            cv2.circle(img, (int(x), int(y)), int(radius),
                (0, 255, 255), 2)
            cv2.circle(img, center, 5, (0, 0, 255), -1)
    # update the points queue
#     pts.appendleft(center)

    cv2.imshow('video',img)
    result.write(img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break #adds delay and looks for key press q to break loop